# 1. Install dependencies

In [ ]:
!pip install -q transformers datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 58.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you

# 2. Example: Run sentiment analysis predictions by using Pipeline
Without specifying a model name, HuggingFace uses the default model, which is `distilbert-base-uncased-finetuned-sst-2-english`.

In [ ]:
# Using pipeline class to make predictions from models available in the Hub in an easy way
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

# 3. Example: Select a sentiment analysis model for our fine-tuning task
We want to pick a base model that is small enough to train in Colab with free resources and see some results after fine-tuning. We will select a couple to evaluate based on likes, number of downloads, and size.
- The `finiteautomata/bertweet-base-sentiment-analysis` model has 135 million parameters. It is based on BERTweet, which is a RoBERTa model pre-trained on 850M English tweets. It is specifically fine-tuned for sentiment analysis on social media text.
- The `distilbert-base-uncased-finetuned-sst-2-english` model is the default model provided by HuggingFace (that we tried out above). It has 66 million parameters and uses DistilBERT, a smaller, faster, and cheaper version of BERT.

Since both models are small and efficient, we'll try out `finiteautomata/bertweet-base-sentiment-analysis`. This model has POSITIVE, NEGATIVE, and NEUTRAL labels, instead of only POSITIVE/NEGATIVE. This time we will pull it into our code from the HuggingFace Models Hub by name. Note that our values will look slightly different since we're using a new model.

**Model Cards:**


*   https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis
*   https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english

**Tip:** If you sign up for a free Hugging Face account, you can interact with their Inference API available on model card pages, allowing you to quickly test sample outputs for various inputs.

In [ ]:
# Using our selected model for sentiment analysis
sentiment_analysis_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
sentiment_analysis_model(data)

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

# 4. Create test data
Now that we have a model in place, let's create some test data that's a little more ambiguous. Add your own!



In [ ]:
test_data = [
    "That was really uncool",
    "I feel unsure",
    "I would rather not work with this company again",
    "I feel you guys did everything you could",
    "The movie was okay, not great but not bad either",
    "I guess it could be worse",
    "The service was decent but could improve",
    "I'm not entirely satisfied with the results",
    "It wasn't what I expected, but it was alright",
    "Is the store open on the weekends?",
    "The food was somewhat edible",
    "I have mixed feelings about this project",
    "How many lines of credit can I open?",
    "The presentation was underwhelming",
    "I suppose it's fine for now",
    "It's not my favorite, but it's acceptable",
    "It's summer weather outside",
    "There are things I like and dislike about it",
    "I'm neither happy nor upset about the outcome",
    "It's passable, though not impressive",
    "The product is okay, but I've seen better",
    "I can tolerate it, but I don't love it",
    "It didn't meet my expectations, but it's not terrible",
    "I'm pretty neutral",
    "Fairly alright",
    "The couch is gray",
    "I hope he eats dirt",
    "Through all the doom and gloom, he survived in the end, though not unscathed",
    "I can't believe how invigorating this experience was, never experienced anything like it",
    "I'm really grateful for the extended hours during the exam season.",
    "I had a hard time finding a quiet spot due to loud conversations.",
    "Are there any penalties for returning books late?"
]

#5. Get a baseline
Log the sentiment labels and scores from the baseline model for each item in the test data. We will use this for a comparison with our fine-tuned model to ensure its performance is improving.

In [ ]:
# Get sentiment predictions
results = sentiment_analysis_model(test_data)

# Print each test data phrase with its corresponding sentiment analysis result
for phrase, result in zip(test_data, results):
    print(f"Sentence: {phrase}\nSentiment: {result}\n")

Sentence: That was really uncool
Sentiment: {'label': 'NEG', 'score': 0.972419261932373}

Sentence: I feel unsure
Sentiment: {'label': 'NEG', 'score': 0.601062536239624}

Sentence: I would rather not work with this company again
Sentiment: {'label': 'NEG', 'score': 0.9815449714660645}

Sentence: I feel you guys did everything you could
Sentiment: {'label': 'POS', 'score': 0.8359615802764893}

Sentence: The movie was okay, not great but not bad either
Sentiment: {'label': 'POS', 'score': 0.7938924431800842}

Sentence: I guess it could be worse
Sentiment: {'label': 'NEU', 'score': 0.7413371801376343}

Sentence: The service was decent but could improve
Sentiment: {'label': 'POS', 'score': 0.9335333108901978}

Sentence: I'm not entirely satisfied with the results
Sentiment: {'label': 'NEG', 'score': 0.9694268703460693}

Sentence: It wasn't what I expected, but it was alright
Sentiment: {'label': 'NEU', 'score': 0.4552016854286194}

Sentence: Is the store open on the weekends?
Sentiment: {'

## Analysis: Areas for improvement
A somewhat subjective analysis:

- Likely Neutral:
  - "I feel unsure" is classified as negative but could be considered more contextually neutral. Its score should reflect more ambiguity.
  - "The movie was okay, not great but not bad either" should be classified as neutral rather than positive.
  - "I have mixed feelings about this project" should be classified as neutral rather than negative.
  - "I'm neither happy nor upset about the outcome" should be classified as neutral rather than negative.
  - "Through all the doom and gloom, he survived in the end, though not unscathed" should likely be neutral rather than positive.
  - "Are there any penalties for returning books late?" should be neutral instead of negative.
- Neutral/Negative:
  - "The product is okay, but I've seen better" should be classified as neutral or negative rather than a strong positive.
  - "I can tolerate it, but I don't love it" may be better as a weaker negative.
  - "The service was decent but could improve" is classified as positive, but has neutral/negative undertones.
- Neutral/Positive:
  - "I can't believe how this experience was, never experienced anything like it" is classified as negative, but has neutral/positive undertones.
  - "Fairly alright" is more neutral/positive than positive.
- Negative:
  - "The food was somewhat edible" is classified as positive, but it suggests a negative sentiment.
  - "It didn't meet my expectations, but it's not terrible" is classified as positive, but it suggests a negative sentiment.

# 6. Fine-tune
## Load a training dataset
Let's try out `yamini0506/hotel_reviews_sentiment_1K`.

Dataset card: https://huggingface.co/datasets/yamini0506/hotel_reviews_sentiment_1K

This dataset comes pre-split into train, val, and test data. This makes it easy for us. However, sometimes we have to split our data ourselves, which can look something like this:
```
### EXAMPLE ###
### SPLITTING A DATASET ###
# Load the dataset
dataset = load_dataset('davanstrien/fake-library-chats-with-sentiment')

# Split the dataset into train and validation sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
```

- **Training dataset**: Used to train the model. The model learns patterns and relationships from this data. Usually the biggest subset of data so it can learn effectively.
- **Validation dataset**: Used to tune the model's hyperparameters and for model selection. It helps in evaluating the model's performance during the training phase and in preventing overfitting by providing feedback on the model's generalization ability. Smaller than training dataset. Often, the data is split into 70-80% for training, 10-15% for validation, and 10-15% for testing.
- **Testing dataset**: Used to evaluate the final model after training and hyperparameter tuning. It provides an unbiased evaluation of the model's performance on unseen data. Smaller than training dataset.

### Tip – Data Integrity:
I referenced another dataset above called `davanstrien/fake-library-chats-with-sentiment`. This dataset looks great upon first glance, but I noticed abnormally high training accuracy/overfitting when I implemented it. Upon further inspection, the data is full of duplicates. It's important to ensure the data you're using is quality and meaningful to get good results. Overfitting = model learns the training data too closely, resulting in it failing to generalize to unseen data.

Dataset Card: https://huggingface.co/datasets/davanstrien/fake-library-chats-with-sentiment

Below, we can see one method for checking a dataset for overlapping samples.

In [ ]:
# Load the dataset
from datasets import load_dataset, Dataset
from collections import Counter

dataset = load_dataset('yamini0506/hotel_reviews_sentiment_1K')

# Inspect the dataset
print(dataset)

# Split the dataset into train and validation sets
train_dataset = dataset['train'].select(range(300))
val_dataset = dataset['validation'].select(range(100))
test_dataset = dataset['test'].select(range(100))

# Inspect the split datasets
print(train_dataset)
print(val_dataset)
print(test_dataset)

# Check for overlapping examples
train_reviews = set([example['total_review'] for example in train_dataset])
test_reviews = set([example['total_review'] for example in test_dataset])

overlap = train_reviews.intersection(test_reviews)
print(f"Number of overlapping reviews: {len(overlap)}")

DatasetDict({
    train: Dataset({
        features: ['total_review', 'sentiment'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['total_review', 'sentiment'],
        num_rows: 600
    })
    test: Dataset({
        features: ['total_review', 'sentiment'],
        num_rows: 300
    })
})
{'total_review': 'Comfortable 1 night stay - Only spent 1 night here. Clean and spacious rooms. Good varied breakfast included. Bed was very comfortable. Would recommend this motel for short stays. Free parking. San Luis Obispo itself is a very nice town!', 'sentiment': 2}
Dataset({
    features: ['total_review', 'sentiment'],
    num_rows: 300
})
Dataset({
    features: ['total_review', 'sentiment'],
    num_rows: 100
})
Dataset({
    features: ['total_review', 'sentiment'],
    num_rows: 100
})
Number of overlapping reviews: 0


## Preprocess the dataset
The key piece here is tokenizing our data.

* Tokenization is a crucial step in NLP that breaks down text into tokens, enabling models to process and learn from the data.
* Tokenizers like AutoTokenizer handle the complex task of converting text into tokens and token IDs, including handling padding, truncation, and special tokens.
* Implementation in practice involves initializing a tokenizer, defining a preprocessing function, and applying it to the dataset.

Each tokenizer has a vocabulary, a mapping of tokens to unique identifiers. During tokenization, text is converted into these identifiers for the model to process.

Example:
```
tokens = tokenizer.tokenize("I love machine learning")
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)  # Output: [1045, 2293, 3698, 4086]
```

In [ ]:
from transformers import AutoTokenizer

# Set up the AutoTokenizer for our base model
tokenizer = AutoTokenizer.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis')

# Write a preprocessing function
def preprocess_function(examples):
    # Tokenize the input texts
    # Use `truncation=True` and `padding=True` in the tokenization process to ensure all input sequences are of consistent length
    tokenized_inputs = tokenizer(examples['total_review'], truncation=True, padding=True)
    # Add labels to the tokenized inputs
    # The Trainer class in transformers library expects 'labels', so we have to remap
    tokenized_inputs['labels'] = examples['sentiment']
    return tokenized_inputs

# Apply the preprocessing function to the train, val, and test datasets
# `batched=True` is a performance optimization that allows the tokenizer to process multiple examples at once
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Inspect the tokenized dataset
print(tokenized_train_dataset)
print(tokenized_val_dataset)
print(tokenized_test_dataset)
print(tokenized_train_dataset[0])
print(tokenized_val_dataset[0])
print(tokenized_test_dataset[0])

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


Dataset({
    features: ['total_review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 300
})
Dataset({
    features: ['total_review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
Dataset({
    features: ['total_review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
{'total_review': 'Comfortable 1 night stay - Only spent 1 night here. Clean and spacious rooms. Good varied breakfast included. Bed was very comfortable. Would recommend this motel for short stays. Free parking. San Luis Obispo itself is a very nice town!', 'sentiment': 2, 'input_ids': [0, 45453, 167, 191, 468, 34, 766, 1665, 167, 191, 3506, 3, 6290, 13, 58508, 20293, 32188, 417, 33047, 2181, 43286, 15800, 4, 7370, 38, 249, 23068, 50132, 4, 1038, 4204, 33, 45440, 19, 1087, 30516, 32188, 950, 2365, 4893, 4, 1753, 13581, 6152, 924, 4499, 2570, 17, 11, 249, 370, 14100, 12, 2, 1, 1, 1, 1, 1, 

## Fine-tune the model using the dataset we loaded

Note: At the top, we've pulled in a `compute_metrics` function to help with an accuracy calculation later on during the final evaluation of our model. We pass this into the TrainingArguments when we initialize the Trainer.


### Key Terms
* **Epoch**: Refers to one complete pass through the entire training dataset. During training, the model updates its weights as it processes the data. An epoch means the model has seen every training example once. Training for multiple epochs allows the model to learn better patterns from the data.
  * Example: If you have 1000 training examples and you train for 5 epochs, your model will have processed 5000 examples in total (though some may be seen more than once).
* **Batch**: A batch is a subset of the training data. Instead of updating the model weights after every single training example, batches allow the model to update weights after processing a set number of examples. Using batches improves computational efficiency and makes the learning process smoother and more stable.
  * Example: If you have 1000 training examples and use a batch size of 100, each epoch will have 10 batches.
* **Loss**: A measure of how well the model's predictions match the actual labels. It quantifies the difference between the predicted outputs and the true outputs. The goal of training is to minimize the loss, making the model's predictions as accurate as possible.
* **Learning Rate**: Hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. A higher learning rate means the model will change weights more quickly, while a lower learning rate means the model will change weights more slowly. The learning rate determines the speed and quality of the convergence to the minimum loss.
* **Weight Decay**:  A regularization technique that involves adding a small penalty to the loss function to discourage large weights. By penalizing large weights, weight decay helps prevent the model from overfitting to the training data. If the model's weights are too large, it might memorize the training data and perform poorly on unseen data. Weight decay helps to keep the weights in check.
* **Overfitting**: Overfitting occurs when a machine learning model captures the noise and details in the training data to such an extent that it negatively impacts the model's performance on new, unseen data. A model that overfits will perform exceptionally well on the training data but poorly on validation or test data because it has memorized the training data rather than learning the underlying patterns.

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_metric

# Initialize the metric to be used for evaluation
metric = load_metric('accuracy')

# Define a function to compute metrics during evaluation
def compute_metrics(p):
    # Get the predictions and labels from the inputs
    predictions, labels = p
    # Convert the predictions to the class with the highest probability
    predictions = predictions.argmax(axis=1)
    # Compute and return the accuracy metric
    return metric.compute(predictions=predictions, references=labels)

# Load the base model using AutoModelForSequenceClassification
# We have 3 sentiment classes (positive, neutral, negative), so pass in 3 labels
model = AutoModelForSequenceClassification.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis', num_labels=3)

# Initialize training arguments
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the model and other outputs
    eval_strategy='epoch',  # Evaluate the model at the end of each epoch
    save_strategy='epoch',  # Save the model at the end of each epoch
    logging_strategy='steps',  # Log training information at regular intervals
    logging_steps=10,  # Log every 10 steps
    learning_rate=2e-5,  # Size of optimization step. Here we have a learning rate of 0.00002
    per_device_train_batch_size=16,  # Number of examples to process per step during training
    per_device_eval_batch_size=32,  # Number of examples to process per step during evaluation
    num_train_epochs=1,  # Number of times the model runs through the entire training data. We're using a single epoch for fast train time
    weight_decay=0.01,  # Penalizes large weights to help prevent overfitting. Set on a logarithmic scale (0.1, 0.01, 0.001...)
)

# Create a Trainer instance with the specified training arguments and datasets
trainer = Trainer(
    model=model,  # The model to be trained
    args=training_args,  # Training arguments defined above
    train_dataset=tokenized_train_dataset,  # The dataset to be used for training
    eval_dataset=tokenized_val_dataset,  # The dataset to be used for evaluation
    tokenizer=tokenizer,  # The tokenizer used for preprocessing the data
    compute_metrics=compute_metrics  # The function to compute metrics during evaluation
)

# Train the model
trainer.train()

# Save the model and tokenizer after training
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

Epoch,Training Loss,Validation Loss,Accuracy
1,0.659600,0.787372,0.680000


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/bpe.codes',
 './results/added_tokens.json')

# 7. Evaluate
Evaluate both the original model and the new model for accuracy.

In [ ]:
# Load the fine-tuned model from the specified directory
new_model = AutoModelForSequenceClassification.from_pretrained('./results')
# Create a pipeline for sentiment analysis using the new model and tokenizer
new_model_pipeline = pipeline("sentiment-analysis", model=new_model, tokenizer=tokenizer)

# Get predictions from the original model
old_results = sentiment_analysis_model(test_data)
# Get predictions from the fine-tuned model
new_results = new_model_pipeline(test_data)

# Compare the results for our test data
for phrase, old_result, new_result in zip(test_data, old_results, new_results):
    print(f"Sentence: {phrase}\nOld Model: {old_result}\nNew Model: {new_result}\n")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Sentence: That was really uncool
Old Model: {'label': 'NEG', 'score': 0.972419261932373}
New Model: {'label': 'NEG', 'score': 0.9366834163665771}

Sentence: I feel unsure
Old Model: {'label': 'NEG', 'score': 0.601062536239624}
New Model: {'label': 'NEG', 'score': 0.5189274549484253}

Sentence: I would rather not work with this company again
Old Model: {'label': 'NEG', 'score': 0.9815449714660645}
New Model: {'label': 'NEG', 'score': 0.9592260122299194}

Sentence: I feel you guys did everything you could
Old Model: {'label': 'POS', 'score': 0.8359615802764893}
New Model: {'label': 'POS', 'score': 0.5431565642356873}

Sentence: The movie was okay, not great but not bad either
Old Model: {'label': 'POS', 'score': 0.7938924431800842}
New Model: {'label': 'NEU', 'score': 0.6108651757240295}

Sentence: I guess it could be worse
Old Model: {'label': 'NEU', 'score': 0.7413371801376343}
New Model: {'label': 'NEU', 'score': 0.7390129566192627}

Sentence: The service was decent but could improve


This is where we test the accuracy of the model using the `compute_metrics` function we defined earlier.

We use the `test` data split from our dataset this time.

In [ ]:
# Load the original model from HuggingFace model hub
# The model has 3 sentiment classes (positive, neutral, negative), so we pass in 3 labels
original_model = AutoModelForSequenceClassification.from_pretrained('finiteautomata/bertweet-base-sentiment-analysis', num_labels=3)

# Initialize training arguments for evaluating the original model
original_training_args = TrainingArguments(
    output_dir='./results_original',
    eval_strategy='epoch',
    per_device_eval_batch_size=16,
)

# Create a Trainer instance for the original model
original_trainer = Trainer(
    model=original_model,
    args=original_training_args,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate the original model using the Trainer
original_model_eval_results = original_trainer.evaluate()
print("Original Model Evaluation Results:", original_model_eval_results)

# Evaluate the fine-tuned model using the Trainer
new_model_eval_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print("New Model Evaluation Results:", new_model_eval_results)

Original Model Evaluation Results: {'eval_loss': 1.0911333560943604, 'eval_accuracy': 0.63, 'eval_runtime': 49.4089, 'eval_samples_per_second': 2.024, 'eval_steps_per_second': 0.142}


New Model Evaluation Results: {'eval_loss': 0.6225600242614746, 'eval_accuracy': 0.7, 'eval_runtime': 40.1125, 'eval_samples_per_second': 2.493, 'eval_steps_per_second': 0.175, 'epoch': 1.0}


## Congrats! You've fine-tuned a sentiment analysis model.
There are many training techniques and hyperparameters that we didn't cover, so I encourage you to go out there and explore other ways of fine-tuning and see if you can improve the model even more.

#### Additional Reading Recommendations:
* https://rentry.org/llm-training
* https://levelup.gitconnected.com/fine-tune-smaller-nlp-models-with-hugging-face-for-specific-use-cases-1745813471dc
* https://addepto.com/blog/rag-vs-fine-tuning-a-comparative-analysis-of-llm-learning-techniques/